2018 / 1 / 31

Wayne Nixalo

# RetinaNet Finetuning -- Dev 4

In [1]:
import keras
import keras.preprocessing.image

# import keras_retinanet.losses
# import keras_retinanet.layers
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
# from keras_retinanet.callbacks import RedirectModel
# from keras_retinanet.models.resnet import ResNet50RetinaNet
# from keras_retinanet.utils.keras_version import check_keras_version
from keras_retinanet.utils.image import preprocess_image, resize_image
from keras_retinanet.models.resnet import custom_objects # <-- what do I need this for?

import tensorflow as tf

import numpy as np
import os

/Users/WayNoxchi/Miniconda3/envs/aersu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

In [21]:
keras.backend.tensorflow_backend.set_session(get_session())

model = keras.models.load_model('data/retinanet-model/resnet50_coco_best_v1.2.2.h5',
                                    custom_objects=custom_objects)

/Users/WayNoxchi/Miniconda3/envs/aersu/lib/python3.6/site-packages/keras/models.py:271: UserWarning: Output "non_maximum_suppression_2" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "non_maximum_suppression_2" during training.
  sample_weight_mode=sample_weight_mode)


## Removing Top Layer from Retinanet

In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
padding_conv1 (ZeroPadding2D)   (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9408        padding_conv1[0][0]              
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu

In [7]:
model.layers[0]

In [14]:
for i,λ in enumerate(model.layers):
    print(f'{i}:  {λ}')

0:  <keras.engine.topology.InputLayer object at 0x18168ce198>
1:  <keras.layers.convolutional.ZeroPadding2D object at 0x18168ce208>
2:  <keras.layers.convolutional.Conv2D object at 0x18168ce390>
3:  <keras_resnet.layers._batch_normalization.BatchNormalization object at 0x18168ce4e0>
4:  <keras.layers.core.Activation object at 0x18168ce6d8>
5:  <keras.layers.pooling.MaxPooling2D object at 0x18168ce668>
6:  <keras.layers.convolutional.Conv2D object at 0x18168ce828>
7:  <keras_resnet.layers._batch_normalization.BatchNormalization object at 0x18168ce978>
8:  <keras.layers.core.Activation object at 0x18168ceac8>
9:  <keras.layers.convolutional.ZeroPadding2D object at 0x18168ceb00>
10:  <keras.layers.convolutional.Conv2D object at 0x18168ceb70>
11:  <keras_resnet.layers._batch_normalization.BatchNormalization object at 0x18168cecc0>
12:  <keras.layers.core.Activation object at 0x18168cee10>
13:  <keras.layers.convolutional.Conv2D object at 0x18168cee48>
14:  <keras.layers.convolutional.Conv2

In [26]:
model.layers[-6:]

So the last 6 layers are the ones I want to cut off. I guess I'm going to want to use the keras_retinanet API to put the RegressBoxes layer back in.

In [28]:
for i in range(6):
    model.layers.pop()

In [29]:
model.layers[-6:]

Looks good. Now to the layers back on with the Functional API. The layers I'm looking to replace:

```
209:  <keras.engine.training.Model object at 0x18169ee940>
210:  <keras.layers.merge.Concatenate object at 0x1816a30048>
211:  <keras.layers.merge.Concatenate object at 0x1816a30080>
212:  <keras_retinanet.layers._misc.RegressBoxes object at 0x1816ab0e48>
213:  <keras.layers.merge.Concatenate object at 0x1816ab0ef0>
214:  <keras_retinanet.layers._misc.NonMaximumSuppression object at 0x1816a300b8>
```

Just saying right now... I don't know how I'm going to be able to replace that `keras.engine.training.Model` layer, "classification_submodel (Model)". Maybe there's a way to replace individual layers, and recompile so Keras will figure out how many in/out pars they should have? We'll see.

In [44]:
from keras.models import Model
from keras.layers import Input, Dense

a = Input(shape=(32,))
b = Dense(32)(a)
model = Model(inputs=a, outputs=b)

In [46]:
# # x = keras.engine.training.Model(model.output_shape, 4)(model)
# last = model.layers[-1].output
# x = keras.engine.training.Model(1,4)(last)

x = keras.engine.training.Model(1,4)(model)

TypeError: Input tensors to a Model must be Keras tensors. Found: 1 (missing Keras metadata).

In [34]:
last

<tf.Tensor 'regression_1/concat:0' shape=(?, ?, 4) dtype=float32>

Okay instead of messing with all this and rebuilding retinanet from scratch just to tweak a part of it, how is it done in the [Boring Detector](https://github.com/lexfridman/boring-detector/blob/master/train.py)?

In [51]:
import keras_retinanet.losses
import keras_retinanet.layers
from keras_retinanet.callbacks import RedirectModel
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.models.resnet import ResNet50RetinaNet
from keras_retinanet.utils.keras_version import check_keras_version

In [55]:
from keras_retinanet.models.resnet import ResNet50RetinaNet
def create_models(num_classes):
    # create "base" model (no NMS)
    image = keras.layers.Input((None, None, 3))
    
    model = ResNet50RetinaNet(image, num_classes=num_classes, weights='imagenet', nms=False)
    training_model = model
    
    # append NMS for prediction only
    classification   = model.outputs[1]
    detections       = model.outputs[2]
    boxes            = keras.layers.Lambda(lambda x: x[:, :, :4])(detections)
    detections       = keras_retinanet.layers.NonMaximumSuppression(name='nms')([boxes, classification, detections])
    prediction_model = keras.models.Model(inputs=model.inputs, outputs=model.outputs[:2] + [detections])
    
    # compile model
    training_model.compile(
        loss={
            'regression'    : keras_retinanet.losses.smooth_l1(),
            'classification':keras_retinanet.losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
    )
    
    return model, training_model, prediction_model

In [61]:
create_models(1)

/Users/WayNoxchi/Miniconda3/envs/aersu/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Output "concatenate_16" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "concatenate_16" during training.


(<keras.engine.training.Model at 0x187a590f60>,
 <keras.engine.training.Model at 0x187a494fd0>)

In [62]:
# just pulling the training model
m1= create_models(1)[0]

/Users/WayNoxchi/Miniconda3/envs/aersu/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Output "concatenate_18" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "concatenate_18" during training.


In [63]:
m1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           (None, None, None, 3 0                                            
__________________________________________________________________________________________________
padding_conv1 (ZeroPadding2D)   (None, None, None, 3 0           input_26[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9408        padding_conv1[0][0]              
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu

 Well.... *THAT* was simple.
 
 So now I'll see if it'll train (freeze Conv layers, train top) and then I'll take a look of whether I want to include NMS or not. NMS is included in the version of RetNet I was using, but I wasn't training it, noly predicted. This here doesn't use NMS during training. Hmm.

## Trying to train the 1-class model

In [67]:
m0,m1,m2 = create_models(1)

/Users/WayNoxchi/Miniconda3/envs/aersu/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Output "concatenate_20" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "concatenate_20" during training.


In [68]:
# model?
m0.output_shape

[(None, None, 4), (None, None, 1), (None, None, 5)]

In [69]:
# training model
m1.output_shape

[(None, None, 4), (None, None, 1), (None, None, 5)]

In [70]:
# prediction model
m2.output_shape

[(None, None, 4), (None, None, 1), (None, None, 5)]

In [71]:
# model?
m0.layers[-6:]

In [72]:
# training model
m1.layers[-6:]

In [73]:
# prediction model
m2.layers[-6:]

In [ ]:
datagen = CSVGenerator(csv_data_file='data/interstage_retinanet_data.csv',
                       csv_class_file='data/interstage_retinanet_classes.csv',
                       image_data_generator=image_gen,
                       batch_size=1)
m1.fit_generator(a)

In [ ]:
model.layers.pop()
for layer in model.layers: layer.trainable = False
    
# recover the output from the last layer in the model and use as input to new Dense layer
last = model.layers[-1].output
x = Dense(train_batches.num_class, activation="softmax")(last)
model = Model(model.input, x)

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])